In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import plotly.express as px
import plotly.graph_objects as go

In [7]:
def basicSVM(X_input, y_input):
    X_train, X_test, y_train, y_test = train_test_split(X_input, y_input, test_size=.1)
    svm_model = SVC(kernel='poly', C=1.0)
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy}')
    print(classification_report(y_test, y_pred))

    trace1 = go.Scatter(x=X_test.iloc[:, 0], y=X_test.iloc[:, 1],
                        mode='markers', marker=dict(color=y_test, colorscale='Viridis', size=10),
                        name='Actual')

    trace2 = go.Scatter(x=X_test.iloc[:, 0], y=X_test.iloc[:, 1],
                        mode='markers', marker=dict(color=y_pred, colorscale='Viridis', size=5, opacity=0.5),
                        name='Predicted')

    layout = go.Layout(title=f'SVM Results (Accuracy: {accuracy:.2f})',
                        xaxis=dict(title='Feature 1'),
                        yaxis=dict(title='Feature 2'))

    fig = go.Figure(data=[trace1, trace2], layout=layout)

    fig.show()

In [8]:
df = pd.read_csv("data_proc/games_basic_master.csv")
df["SCORE_DIFF"] = df["TEAM1_SCORE"] - df["TEAM2_SCORE"]
df.dropna(inplace=True)
basicSVM(df[["TEAM1_ID", "TEAM2_ID", "away_prob", "away_spread", "away_total"]],
                df[["SCORE_DIFF"]])

/Users/amanda/miniconda3/envs/ml_hw3/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.05181347150259067
              precision    recall  f1-score   support

       -43.0       0.00      0.00      0.00         1
       -42.0       0.00      0.00      0.00         1
       -39.0       0.00      0.00      0.00         2
       -36.0       0.00      0.00      0.00         1
       -35.0       0.00      0.00      0.00         1
       -34.0       0.00      0.00      0.00         1
       -33.0       0.00      0.00      0.00         2
       -32.0       0.00      0.00      0.00         4
       -31.0       0.00      0.00      0.00         4
       -30.0       0.00      0.00      0.00         5
       -29.0       0.00      0.00      0.00         6
       -28.0       0.00      0.00      0.00         2
       -27.0       0.00      0.00      0.00         7
       -26.0       0.00      0.00      0.00         4
       -25.0       0.00      0.00      0.00        10
       -24.0       0.00      0.00      0.00         2
       -23.0       0.00      0.00      0.00        

/Users/amanda/miniconda3/envs/ml_hw3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amanda/miniconda3/envs/ml_hw3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/amanda/miniconda3/envs/ml_hw3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,